In [1]:
import numpy
from context import vaeqst

# Random Clifford Circuit

## RandomCliffordGate

`RandomClifordGate(*qubits)` represents a random Clifford gate acting on a set of qubits. There is no further parameter to specify, as it is not any particular gate, but a placeholder for a generic random Clifford gate.

**Parameters**
- `*qubits`: indices of the set of qubits on which the gate acts on.

Example:

In [3]:
gate = vaeqst.RandomCliffordGate(0,1)
gate

[0,1]

`RandomCliffordGate.random_clifford_map()` evokes a random sampling of the Clifford unitary, return in the form of operator mapping table $M$ and the corresponding sign indicator $h$. Such that under the mapping, any Pauli operator $\sigma_g$  specified by the binary representation $g$ (and localized within the gate support) gets mapped to
$$\sigma_g \to \prod_{i=1}^{2n} (-)^{h_i}\sigma_{M_i}^{g_i}.$$
The binary representation is in the $g=(x_0,z_0,x_1,z_1,\cdots)$ basis.

In [4]:
gate.random_clifford_map()

(array([[1, 1, 1, 1],
        [0, 0, 1, 0],
        [1, 0, 1, 0],
        [0, 1, 1, 0]]), array([0, 1, 0, 1]))

## RandomCliffordLayer

`RandomCliffordLayer(*gates)` represents a layer of random Clifford gates. 

**Parameters:**
* `*gates`: quantum gates contained in the layer.

The gates in the same layer should not overlap with each other (all gates need to commute). To ensure this, we do not manually add gates to the layer, but using the higher level function `.gate()` provided by `RandomCliffordCircuit` (see discussion later).

Example:

In [2]:
layer = vaeqst.RandomCliffordLayer(vaeqst.RandomCliffordGate(0,1),vaeqst.RandomCliffordGate(3,5))
layer

|[0,1][3,5]|

It hosts a list of gates:

In [6]:
layer.gates

[[0,1], [3,5]]

Given the total number of qubits $N$, the layer can sample the Clifford unitary (as product of each gate) $U=\prod_{a}U_a$, and represent it as a single operator mapping (because gates do not overlap, so they maps operators in different supports independently).

In [5]:
layer.random_clifford_map(6)

(array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]]),
 array([0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0]))

## RandomCliffordCircuit

`RandomCliffordCircuit()` represents a quantum circuit of random Clifford gates.

### Methods

#### Construct the  Circuit

Example: create a random Clifford circuit.

In [2]:
circ = vaeqst.RandomCliffordCircuit()

Use `.gate(*qubits)` to add random Clifford gates to the circuit.

In [3]:
circ.gate(0,1)
circ.gate(2,4)
circ.gate(1,4)
circ.gate(0,2)
circ.gate(3,5)
circ.gate(3,4)
circ

|[3,4]|
|[1,4][0,2]|
|[0,1][2,4][3,5]|

Gates will automatically arranged into layers. Each new gate added to the circuit will commute through the layers if it is not blocked by the existing gates.

If the number of qubits `.N` is not explicitly defined, it will be dynamically infered from the circuit width, as the largest qubit index of all gates + 1.

In [4]:
circ.N

6

#### Navigate in the Circuit

`.layers_forward()` and `.layers_backward()` provides two generators to iterate over layers in forward and backward order resepctively.

In [4]:
list(circ.layers_forward())

[|[0,1][2,4][3,5]|, |[1,4][0,2]|, |[3,4]|]

In [5]:
list(circ.layers_backward())

[|[3,4]|, |[1,4][0,2]|, |[0,1][2,4][3,5]|]

`.first_layer` and `.last_layer` points to the first and the last layers.

In [5]:
circ.first_layer

|[0,1][2,4][3,5]|

In [6]:
circ.last_layer

|[3,4]|

Use `.next_layer` and `.prev_layer` to move forward and backward.

In [7]:
circ.first_layer.next_layer, circ.last_layer.prev_layer

(|[1,4][0,2]|, |[1,4][0,2]|)

Locate a gate in the circuit.

In [8]:
circ.first_layer.next_layer.next_layer.gates[0]

[3,4]

#### Apply Circuit to State

`.forward(state)` and `.backward(state)` applies the circuit to transform the state forward / backward. 
* Each call will sample a new random realization of the random Clifford circuit.
* The transformation will create a new state, the original state remains untouched.

In [6]:
rho = vaeqst.StabilizerState(6, r=0)
rho

StabilizerState(
 +ZIIIII
 +IZIIII
 +IIZIII
 +IIIZII
 +IIIIZI
 +IIIIIZ)

In [7]:
circ.forward(rho)

StabilizerState(
 -XIXIII
 -IYIIII
 +ZYYZXI
 -IIIZIY
 -ZIYIII
 -IIIYYX)

In [8]:
circ.backward(rho)

StabilizerState(
 -ZXIIII
 +IIYIZI
 -IIZIYI
 +XZYIZZ
 +IIIIIZ
 -IIIXIZ)

#### POVM

`.povm(nsample)` provides a generator to sample $n_\text{sample}$ from the prior POVM based on the circuit by back evolution.

In [15]:
list(circ.povm(3))

[StabilizerState(
  +IIZIYI
  +YZYIZI
  +ZYIIII
  +IIYZZX
  +YXZXII
  +IIIIIX), StabilizerState(
  +ZYIIII
  +YXYIZI
  +IIIIZI
  +XXZIZI
  -IIIZIZ
  +IIIZII), StabilizerState(
  -YXYIZI
  -XYZIYI
  +IYZIII
  +IIIZIX
  -IIZIYI
  -IIIZII)]

## BrickWallRCC

`BrickWallRCC(N, depth)` is a subclass of `RandomCliffordCircuit`. It represents the circuit with 2-qubit gates arranged following a brick wall pattern.

In [68]:
circ = vaeqst.BrickWallRCC(16,2)
circ

|[1,2][3,4][5,6][8,7][9,10][11,12][13,14][0,15]|
|[0,1][2,3][4,5][6,7][8,9][10,11][12,13][14,15]|

Create an inital state as a computational basis state.

In [69]:
rho = vaeqst.StabilizerState(16, r=0)
rho

StabilizerState(
 +ZIIIIIIIIIIIIIII
 +IZIIIIIIIIIIIIII
 +IIZIIIIIIIIIIIII
 +IIIZIIIIIIIIIIII
 +IIIIZIIIIIIIIIII
 +IIIIIZIIIIIIIIII
 +IIIIIIZIIIIIIIII
 +IIIIIIIZIIIIIIII
 +IIIIIIIIZIIIIIII
 +IIIIIIIIIZIIIIII
 +IIIIIIIIIIZIIIII
 +IIIIIIIIIIIZIIII
 +IIIIIIIIIIIIZIII
 +IIIIIIIIIIIIIZII
 +IIIIIIIIIIIIIIZI
 +IIIIIIIIIIIIIIIZ)

Backward evolve the state to obtain the measurement operator.

In [71]:
circ.backward(rho)

StabilizerState(
 -XZIIIIIIIIIIIIII
 -IIXIIIIIIIIIIIII
 -ZXIIIIIIIIIIIIII
 +IIXZXZIIIIIIIIII
 +IIXZIIIIIIIIIIII
 -IIIIXIXIIIIIIIII
 -IIIIIIXIIIIIIIII
 -IIIIIIIIIYIIIIII
 -IIIIIIXZIYIIIIII
 -IIIIIIIIYIZIIIII
 -IIIIIIIIIIZIIIII
 -IIIIIIIIIIIYXIII
 +IIIIIIIIIIZXZYII
 -IIIIIIIIIIIIXXII
 +IIIIIIIIIIIIIIYI
 -IIIIIIIIIIIIIIIY)

## OnSiteRCC

`OnSiteRCC(N)` is a subclass of `RandomCliffordCircuit`. It represents the circuit of a single layer of on-site Clifford gates. It can be used to generate random Pauli states.

In [6]:
circ = vaeqst.OnSiteRCC(16)
circ

|[0][1][2][3][4][5][6][7][8][9][10][11][12][13][14][15]|

In [8]:
rho = vaeqst.StabilizerState(16, r=0)
circ.backward(rho)

StabilizerState(
 +XIIIIIIIIIIIIIII
 +IXIIIIIIIIIIIIII
 +IIZIIIIIIIIIIIII
 -IIIZIIIIIIIIIIII
 +IIIIYIIIIIIIIIII
 -IIIIIYIIIIIIIIII
 -IIIIIIYIIIIIIIII
 -IIIIIIIYIIIIIIII
 +IIIIIIIIXIIIIIII
 -IIIIIIIIIYIIIIII
 -IIIIIIIIIIXIIIII
 -IIIIIIIIIIIZIIII
 -IIIIIIIIIIIIYIII
 +IIIIIIIIIIIIIYII
 +IIIIIIIIIIIIIIYI
 -IIIIIIIIIIIIIIIY)

## GlobalRCC

`GlobalRCC(N)` is a subclass of `RandomCliffordCircuit`. It represents the circuit consists of a global Clifford gate. It can be used to generate Clifford states.

In [9]:
circ = vaeqst.GlobalRCC(16)
circ

|[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]|

In [10]:
rho = vaeqst.StabilizerState(16, r=0)
circ.backward(rho)

StabilizerState(
 +ZZZIXIZXYZXXIYYY
 -ZYXIYZZIYYXZIYXI
 -XZXIZIXZXZXIZZYZ
 +XIXXXZZZIIYIYIYI
 +ZYZXXYZIXIIXXYYY
 +IZIIYYYXXXZZXIII
 -ZXIZZZYYYYZZIXYI
 -IXIYIIYZYYYZYYYY
 +IZIYXZYIYYXZZIYZ
 +IIZZIXZYZZZYXZYZ
 -YZXIIZXXIYZYXXXY
 -ZIXYZZIIXYIYXYYZ
 -IYZXXIYXYXYIIZXY
 -ZIYZZXXIIZXYXZYX
 +YXYIZYYZIIXYZIZY
 -YYYYYIZXYZZIIYIX)